In [84]:
# importing pandas package 
import pandas as pd 
import numpy as np
import heapq
from collections import Counter
from sklearn.metrics.pairwise import cosine_similarity
from sklearn import datasets, model_selection
from sklearn.metrics import classification_report	


In [85]:
# making data frame from csv file 
gros_data = pd.read_csv("groceries.csv",sep=";") 
gros_data.head(20)


,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
0,1,75,male,married,primary,20000,very_good,3,retired,"citrus fruit,semi-finished bread,margarine,rea..."
1,2,61,female,single,secondary,28000,good,1,housemaid,"tropical fruit,yogurt,coffee"
2,3,32,male,single,secondary,34000,very_good,1,blue-collar,whole milk
3,4,62,male,married,primary,31000,very_good,3,blue-collar,"pip fruit,yogurt,cream cheese,meat spreads"
4,5,66,female,married,secondary,19000,good,3,retired,"other vegetables,whole milk,condensed milk,lon..."
5,6,55,female,single,secondary,35000,very_good,1,unemployed,"whole milk,butter,yogurt,rice,abrasive cleaner"
6,7,23,female,married,tertiary,21000,good,3,housemaid,rolls/buns
7,8,26,female,single,secondary,30000,good,2,blue-collar,"other vegetables,UHT-milk,rolls/buns,bottled b..."
8,9,29,female,married,secondary,32000,very_good,3,blue-collar,potted plants
9,10,57,female,married,secondary,26000,good,3,entrepreneur,"whole milk,cereals"


In [86]:
gros_data.dtypes

Customer_ID              int64
Age                     object
Sex                     object
Marital_Status          object
Education               object
Income                  object
Customer_Rating         object
Persons_in_Household     int64
Occupation              object
Groceries               object
dtype: object

In [91]:
gros_data = gros_data.replace(r'^\s*$', np.nan, regex=True)
gros_data["Age"] = pd.to_numeric(gros_data["Age"],downcast='integer')
gros_data["Income"] = pd.to_numeric(gros_data["Income"],downcast='integer')
gros_data["Persons_in_Household"] = pd.to_numeric(gros_data["Persons_in_Household"])

gros_data["Sex"] = gros_data["Sex"].astype('category')

gros_data.dtypes

Customer_ID                int64
Age                      float64
Sex                     category
Marital_Status            object
Education                 object
Income                   float64
Customer_Rating           object
Persons_in_Household       int64
Occupation                object
Groceries                 object
dtype: object

In [93]:

gros_data.fillna(gros_data.mean().astype(int), inplace=True)
gros_data["Age"] = gros_data["Age"].astype(np.int64) 
gros_data["Income"] = gros_data["Income"].astype(np.int64) 

gros_data.head(20)



,Customer_ID,Age,Sex,Marital_Status,Education,Income,Customer_Rating,Persons_in_Household,Occupation,Groceries
0,1,75,male,married,primary,20000.0,very_good,3,retired,"citrus fruit,semi-finished bread,margarine,rea..."
1,2,61,female,single,secondary,28000.0,good,1,housemaid,"tropical fruit,yogurt,coffee"
2,3,32,male,single,secondary,34000.0,very_good,1,blue-collar,whole milk
3,4,62,male,married,primary,31000.0,very_good,3,blue-collar,"pip fruit,yogurt,cream cheese,meat spreads"
4,5,66,female,married,secondary,19000.0,good,3,retired,"other vegetables,whole milk,condensed milk,lon..."
5,6,55,female,single,secondary,35000.0,very_good,1,unemployed,"whole milk,butter,yogurt,rice,abrasive cleaner"
6,7,23,female,married,tertiary,21000.0,good,3,housemaid,rolls/buns
7,8,26,female,single,secondary,30000.0,good,2,blue-collar,"other vegetables,UHT-milk,rolls/buns,bottled b..."
8,9,29,female,married,secondary,32000.0,very_good,3,blue-collar,potted plants
9,10,57,female,married,secondary,26000.0,good,3,entrepreneur,"whole milk,cereals"
